In [ ]:
#defines a function segmentImages that takes a list of filenames of jpeg files, and segments
#to produce two masks of the same size for each of them, one mask for birds, one mask for squirrels.

In [ ]:
pip install pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
  import tensorflow as tf
  import tensorflow_hub as tfhub
  import numpy as np
  from PIL import Image
  import cv2

In [ ]:
from google.colab import drive

#for authorisation
drive.mount('/content/drive')

#Drive files in "/content/drive/My Drive".
!ls "/content/drive/My Drive"

Mounted at /content/drive
 bird_birdfeeder_random
 BirdfeederModel
'Copy of 4131.jpg'
'Copy of hw3_dataset.tfrecord'
'Essay Drafts'
 hw3_dataset.tfrecord
'joinedForSquirrels MLHW3'
'MLHW 3'
 MLHW3.3
'ML Lab 1'
'ML Lab 2'
'ML Lab 3'
'Physics Project'
'Physics Scripts'
 Pics
'pictures of squirrels at birdfeeders - Google Searc'
 segmentImages
 squirrel_birdfeeder_random
 squirrel.jpg
 squirrelsGone
 yolov3.cfg
 yolov3.weights


In [ ]:
def segmentImages_Bird(fileList):

  masks=[]

  model_coco = tfhub.load('https://tfhub.dev/google/HRNet/coco-hrnetv2-w48/1')

  for bird in fileList:
    image_path = bird
#    image_path = '/content/drive/My Drive/squirrel_birdfeeder_random/' + bird

    inBirdIm=tf.io.decode_jpeg(tf.io.read_file(image_path),channels=3)

    resp_bird=model_coco.predict([tf.cast(inBirdIm,tf.float32)/255.])
    myBirdIm=resp_bird.numpy()

    test=myBirdIm[:,:,:,17].max()

    if test>.9:

      maskIm_bird=np.ones_like(inBirdIm)
      mask_bird=tf.squeeze(resp_bird)[:,:,17]>.4

      if len(inBirdIm[mask_bird])>0:
        maskIm_bird=inBirdIm[mask_bird].numpy().max()*maskIm_bird

      maskIm_bird[mask_bird]=inBirdIm[mask_bird]

      mask_image = Image.fromarray(maskIm_bird)

      masks.append(mask_image)

  return masks

In [ ]:
path='/content/drive/My Drive/bird_birdfeeder_random/'
BirdMasks=segmentImages_Bird(path)

In [ ]:
#Now for squirrels:

def segmentImages_Squirrel(fileList):

  import tensorflow as tf
  import tensorflow_hub as tfhub
  import numpy as np
  from PIL import Image
  import cv2

  masks=[]

  model_coco = tfhub.load('https://tfhub.dev/google/HRNet/coco-hrnetv2-w48/1')

  #It is specified in the original problem to create masks of a standard size, but that was when both masks were used for the same image.
  #I had resized the masks to all be the same size across different pictures, but remembering what successful outputs from class,
  #I don't think I'm supposed to be doing that. So, I won't.

  for sqrl in fileList:

    image_path=sqrl

#    image_path = '/content/drive/My Drive/squirrel_birdfeeder_random/' + sqrl

    inSquirrelIm=tf.io.decode_jpeg(tf.io.read_file(image_path),channels=3)

    resp_squirrel=model_coco.predict([tf.cast(inSquirrelIm,tf.float32)/255.])
    mySquirrelIm=resp_squirrel.numpy()

    test=mySquirrelIm[:,:,:,18].max()

    if test>.7:

      maskIm_squirrel=np.ones_like(inSquirrelIm)
      mask_squirrel=tf.squeeze(resp_squirrel)[:,:,18]>.4

      if len(inSquirrelIm[mask_squirrel])>0:
        maskIm_squirrel=inSquirrelIm[mask_squirrel].numpy().max()*maskIm_squirrel

      maskIm_squirrel[mask_squirrel]=inSquirrelIm[mask_squirrel]

      mask_image = Image.fromarray(maskIm_squirrel)

      masks.append(mask_image)

  return masks

In [ ]:
path='/content/drive/My Drive/squirrel_birdfeeder_random/'
SquirrelMasks=segmentImages_Squirrel(path)